In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

^C


In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [3]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        
        # Temperature array
        self.observation_space = Box(low=np.array([-7]), high=np.array([20]))
        # Set start temp
        
        # Set shower length
        self.z1=0
        self.z2=0
        self.ground_length = 11
        self.pointer1=2
        self.pointer2=0
        self.lidarPointer=3
        self.ground=[0,0,0,0,1,3,5,3,1,0,0,0,0,0,0,0]
        self.lidar=self.ground[self.lidarPointer]
        self.state = 0
        self.action_space = Discrete(3)
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        if action==0:
            if(self.z1<=7 and self.z1>=-7 and self.z2<=7 and self.z2>=-7):
                self.z1-=0.2
                self.z2+=0.2
        if action==1:
            self.z1-=0
            self.z2+=0
        if action==2:
            if(self.z1<=7 and self.z1>=-7 and self.z2<=7 and self.z2>=-7):
                self.z1+=0.2
                self.z2-=0.2

     
        #print(self.state)
        self.state += (self.z1 +self.ground[self.pointer1]) - (self.z2 +self.ground[self.pointer2])
        # Reduce shower length by 1 second
        self.ground_length -= 1 
        self.pointer1+=1
        self.pointer2+=1
        self.lidarPointer+=1
        self.lidar=self.ground[self.lidarPointer]
        #self.state[1] = self.lidar
        
        # Calculate reward
        if self.state >=-3 and self.state <=3: 
            reward =0.1 
        else: 
            reward = -0.1 
        
        # Check if shower is done
        if self.ground_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        #print(self.state,self.z1,self.z2,self.ground[self.pointer1],self.ground[self.pointer2])
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        
        # Set shower length
        self.z1=0
        self.z2=0
        self.ground_length = 11
        self.pointer1=2
        self.pointer2=0
        self.lidarPointer=3
        self.lidar=self.ground[self.lidarPointer]
        self.state = 0
        return self.state

In [4]:
env = ShowerEnv()

In [5]:
env.action_space.sample()

0

In [6]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        env.render()
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-0.09999999999999998
Episode:2 Score:0.30000000000000004
Episode:3 Score:-0.5
Episode:4 Score:-0.5
Episode:5 Score:-0.5
Episode:6 Score:0.5
Episode:7 Score:-0.09999999999999998
Episode:8 Score:-0.09999999999999998
Episode:9 Score:-0.5
Episode:10 Score:-0.5


In [7]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

ERROR: tensorflow-gpu 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow-gpu 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.7.0 which is incompatible.
ERROR: tensorflow-gpu 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.3.0 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\chinn\\anaconda3\\envs\\tensorflow1\\lib\\site-packages\\tensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:


In [8]:
states = env.observation_space.shape
actions = env.action_space.n

In [9]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu',input_shape=states))    
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [16]:
del model

In [17]:
model = build_model(states, actions)

In [18]:
model.output_shape
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                48        
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [13]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [19]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [20]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=5000, visualize=False, verbose=1)

Training for 5000 steps ...
Interval 1 (0 steps performed)
 5000/10000 [==============>...............] - ETA: 30s - reward: -0.0033- ETA: 30s - reward: done, took 30.107 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -0.300, steps: 11
Episode 2: reward: -0.300, steps: 11
Episode 3: reward: -0.300, steps: 11
Episode 4: reward: -0.300, steps: 11
Episode 5: reward: -0.300, steps: 11
Episode 6: reward: -0.300, steps: 11
Episode 7: reward: -0.300, steps: 11
Episode 8: reward: -0.300, steps: 11
Episode 9: reward: -0.300, steps: 11
Episode 10: reward: -0.300, steps: 11
Episode 11: reward: -0.300, steps: 11
Episode 12: reward: -0.300, steps: 11
Episode 13: reward: -0.300, steps: 11
Episode 14: reward: -0.300, steps: 11
Episode 15: reward: -0.300, steps: 11
Episode 16: reward: -0.300, steps: 11
Episode 17: reward: -0.300, steps: 11
Episode 18: reward: -0.300, steps: 11
Episode 19: reward: -0.300, steps: 11
Episode 20: reward: -0.300, steps: 11
Episode 21: reward: -0.300, steps: 11
Episode 22: reward: -0.300, steps: 11
Episode 23: reward: -0.300, steps: 11
Episode 24: reward: -0.300, steps: 11
Episode 25: reward: -0.300, steps: 11
Episode 26: reward: -0.300, st

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=False)

Testing for 15 episodes ...
Episode 1: reward: -0.300, steps: 11
Episode 2: reward: -0.300, steps: 11
Episode 3: reward: -0.300, steps: 11
Episode 4: reward: -0.300, steps: 11
Episode 5: reward: -0.300, steps: 11
Episode 6: reward: -0.300, steps: 11
Episode 7: reward: -0.300, steps: 11
Episode 8: reward: -0.300, steps: 11
Episode 9: reward: -0.300, steps: 11
Episode 10: reward: -0.300, steps: 11
Episode 11: reward: -0.300, steps: 11
Episode 12: reward: -0.300, steps: 11
Episode 13: reward: -0.300, steps: 11
Episode 14: reward: -0.300, steps: 11
Episode 15: reward: -0.300, steps: 11
